# Homework 3 - Using DP for object reconstruction from shadows

In this homework we use mitsuba studio 3 (3.6.4), with python 3.12. 

In [37]:
import mitsuba as mi
import os
import drjit as dr
import numpy as np
from tqdm import tqdm

# See the variants available on the Mac M3 Pro
mi.variants()

['scalar_rgb',
 'scalar_spectral',
 'scalar_spectral_polarized',
 'llvm_ad_rgb',
 'llvm_ad_mono',
 'llvm_ad_mono_polarized',
 'llvm_ad_spectral',
 'llvm_ad_spectral_polarized']

In [38]:
# We set the LLVM AutoDiff - MacOS
mi.set_variant('llvm_ad_rgb')

We open a 3D scene in Mitsuba XML format and render it.

In [29]:
# This is some macos specific - DRJIT LLVM lib path, which needs to be exported
os.environ['DRJIT_LIBLLVM_PATH'] = '/opt/homebrew/opt/llvm/lib/libLLVM.dylib'

sphere_scene = mi.load_file('sphere-scene.xml')
sphere_img = mi.render(sphere_scene, spp=16)

We can view the image with the Bitmap class

In [30]:
mi.util.convert_to_bitmap(sphere_img)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [900, 800],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 2.06 MiB of image data ]
]

We also open our reference scene

In [32]:
cube_scene = mi.load_file('cube-scene.xml')
cube_img = mi.render(cube_scene, spp=16)

mi.util.convert_to_bitmap(cube_img)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [900, 800],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 2.06 MiB of image data ]
]

We can traverse our scene to find the shadow object cast on the wall, this is the object against which we will optimize.

In [33]:
params = mi.traverse(sphere_scene)
print(params)

SceneParameters[
  --------------------------------------------------------------------------------------------
  Name                                     Flags    Type           Parent
  --------------------------------------------------------------------------------------------
  default-bsdf.brdf_0.reflectance.value    ∂        Float          UniformSpectrum
  elm__1.near_clip                                  float          PerspectiveCamera
  elm__1.far_clip                                   float          PerspectiveCamera
  elm__1.shutter_open                               float          PerspectiveCamera
  elm__1.shutter_open_time                          float          PerspectiveCamera
  elm__1.film.size                                  ScalarVector2u HDRFilm
  elm__1.film.crop_size                             ScalarVector2u HDRFilm
  elm__1.film.crop_offset                           ScalarPoint2u  HDRFilm
  elm__1.x_fov                             ∂, D     Float          Pers

In [35]:
# Since we're optimizing a PLY mesh sphere, we'll optimize its vertices directly
params.keep(['sphere.vertex_positions'])

# Store initial vertex positions for regularization
initial_vertices = dr.detach(params['sphere.vertex_positions'])

We now set up the optimizer and the optimization loop

In [ ]:
opt = mi.ad.Adam(params=params, lr=0.01)

# Pre-render the reference (cube) image and save it for comparison
cube_img = mi.render(cube_scene, spp=64)
mi.util.write_bitmap("reference_cube_start.exr", cube_img)

# Save initial sphere image for comparison
initial_sphere_img = mi.render(sphere_scene, params=params, spp=64)
mi.util.write_bitmap("initial_sphere.exr", initial_sphere_img)

# Verify we're seeing initial differences
diff_img = dr.abs(initial_sphere_img - cube_img)
mi.util.write_bitmap("initial_difference.exr", diff_img)

for i in tqdm(range(10)):
    # IMPORTANT: Reset the parameters to check if they're actually being applied
    if i % 50 == 0 and i > 0:
        # Save current vertex positions 
        current_vertices = dr.detach(params['sphere.vertex_positions'])
        
        # Calculate vertex displacement to check if optimization is working
        displacement = dr.sum(dr.abs(current_vertices - initial_vertices))
        print(f"Total vertex displacement after {i} iterations: {float(displacement)}")
    
    # Explicitly enable gradients
    dr.enable_grad(params['sphere.vertex_positions'])
    
    # Render sphere scene with differentiable params
    sphere_img = mi.render(sphere_scene, params=params, spp=64)
    
    # Calculate MSE manually with a multiplier to amplify the gradient signal
    diff = sphere_img - cube_img
    mse_loss = dr.mean(diff * diff) 
    
    # MUCH WEAKER regularization to allow more deformation
    reg_strength = 0.0001  # Reduced from 0.001
    reg_loss = reg_strength * dr.sum(dr.square(params['sphere.vertex_positions'] - initial_vertices))
    
    # Combined loss
    loss = mse_loss + reg_loss
    
    # Backpropagate
    dr.backward(loss)
    
    # Step optimizer with larger step size
    opt.step()
    
    # CRUCIAL: Make sure updated parameters are applied
    params.update()
    
    # Print progress and save intermediate results
    if i % 10 == 0:
        loss_val = float(dr.detach(mse_loss))
        print(f"Iteration {i}, MSE Loss: {loss_val}")
        
        # Save current render
        mi.util.write_bitmap(f"progress_{i:04d}.exr", sphere_img)
        
        # Save the difference image to visually track progress
        diff_img = dr.abs(sphere_img - cube_img)
        mi.util.write_bitmap(f"diff_{i:04d}.exr", diff_img)

# Final high-quality render to check if changes were applied
print("Rendering final result...")
final_sphere_img = mi.render(sphere_scene, params=params, spp=1024)
mi.util.write_bitmap("final_optimized.exr", final_sphere_img)
mi.util.write_bitmap("reference_cube_final.exr", mi.render(cube_scene, spp=1024))

# Save final difference image
final_diff = dr.abs(final_sphere_img - mi.render(cube_scene, spp=1024))
mi.util.write_bitmap("final_difference.exr", final_diff)

# Check if optimization made any difference
print(f"Final difference: {float(dr.mean(final_diff))}")


  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
print(params['sphere.vertex_positions'])

[-0.00662142, 1.97278, -0.0369338, .. 26409 skipped .., -0.0110292, 1.97156, -0.0432549]
